In [132]:
import pandas as pd
import requests
from bs4 import BeautifulSoup as bs
from splinter import Browser
from datetime import datetime
import json

In [2]:
file = "MasterData_Wilson.xlsx"
pd_file = pd.read_excel(file)
bakery_df = pd.DataFrame(pd_file)
bakery_df.head()

,date,time,year,MonthN,Month,WeekN,Day,WeekD,H,Min,...,Price,Quantity,Modifiers,Subtotal,Discounts,Net Total,Tax,Total Due,Unnamed: 29,Unnamed: 30
0,2016-07-25,2016-07-25 09:04:00,2016.0,7.0,Jul,31.0,25.0,Mon,9.0,4.0,...,2.25,1,0,2.25,0,2.25,0.05,2.3,NaN,NaN
1,2016-07-25,2016-07-25 09:28:00,2016.0,7.0,Jul,31.0,25.0,Mon,9.0,28.0,...,2,2,0,4,0,4,0.09,4.09,NaN,NaN
2,2016-07-25,2016-07-25 09:28:00,2016.0,7.0,Jul,31.0,25.0,Mon,9.0,28.0,...,3.75,1,0,3.75,0,3.75,0.38,4.13,NaN,NaN
3,2016-07-25,2016-07-25 09:28:00,2016.0,7.0,Jul,31.0,25.0,Mon,9.0,28.0,...,3.8,1,0,3.8,0,3.8,0.09,3.89,NaN,NaN
4,2016-07-25,2016-07-25 09:29:00,2016.0,7.0,Jul,31.0,25.0,Mon,9.0,29.0,...,3.5,1,0,3.5,0,3.5,0.08,3.58,NaN,NaN


In [3]:
bakery_df.columns

Index(['date', 'time', 'year', 'MonthN', 'Month', 'WeekN', 'Day', 'WeekD', 'H',
       'Min', '30min', 'Rtime', '30time', 'H.5', 'Operation', 'Line Item',
       'Department', 'Category', 'Supplier', 'Supplier Code', 'Cost', 'Price',
       'Quantity', 'Modifiers', 'Subtotal', 'Discounts', 'Net Total', 'Tax',
       'Total Due', 'Unnamed: 29', 'Unnamed: 30'],
      dtype='object')

In [11]:
bakery_df['Unnamed: 29'].value_counts()

Sat    1
2      1
Name: Unnamed: 29, dtype: int64

In [10]:
bakery_df.loc[(bakery_df['Unnamed: 29'] == 'Sat') | (bakery_df['Unnamed: 29'] == 2), :]

,date,time,year,MonthN,Month,WeekN,Day,WeekD,H,Min,...,Price,Quantity,Modifiers,Subtotal,Discounts,Net Total,Tax,Total Due,Unnamed: 29,Unnamed: 30
393033,2018-07-22,2018-07-22 11:18:00,2018.0,7.0,Jul,29.0,22.0,Sun,11.0,18.0,...,3.8,2,0,7.6,0,7.6,0.18,7.78,2,2.0
393034,2018-07-22,2018-07-22 11:18:00,2018.0,7.0,Jul,29.0,22.0,Sun,11.0,18.0,...,3.5,4,0,14,0,14,0.32,14.32,Sat,NaN


In [12]:
bakery_df['Unnamed: 30'].value_counts()

2.0    1
Name: Unnamed: 30, dtype: int64

In [15]:
bakery_df = bakery_df[['date', 'time', 'year', 'MonthN', 'Month', 'WeekN', 'Day', 'WeekD', 'H',
       'Min', '30min', 'Rtime', '30time', 'H.5', 'Operation', 'Line Item',
       'Department', 'Category', 'Supplier', 'Supplier Code', 'Cost', 'Price',
       'Quantity', 'Modifiers', 'Subtotal', 'Discounts', 'Net Total', 'Tax',
       'Total Due']]

In [16]:
bakery_df.head()

,date,time,year,MonthN,Month,WeekN,Day,WeekD,H,Min,...,Supplier Code,Cost,Price,Quantity,Modifiers,Subtotal,Discounts,Net Total,Tax,Total Due
0,2016-07-25,2016-07-25 09:04:00,2016.0,7.0,Jul,31.0,25.0,Mon,9.0,4.0,...,MACCHO,1,2.25,1,0,2.25,0,2.25,0.05,2.3
1,2016-07-25,2016-07-25 09:28:00,2016.0,7.0,Jul,31.0,25.0,Mon,9.0,28.0,...,00:00:00,0,2,2,0,4,0,4,0.09,4.09
2,2016-07-25,2016-07-25 09:28:00,2016.0,7.0,Jul,31.0,25.0,Mon,9.0,28.0,...,CBC,0,3.75,1,0,3.75,0,3.75,0.38,4.13
3,2016-07-25,2016-07-25 09:28:00,2016.0,7.0,Jul,31.0,25.0,Mon,9.0,28.0,...,CRC,0,3.8,1,0,3.8,0,3.8,0.09,3.89
4,2016-07-25,2016-07-25 09:29:00,2016.0,7.0,Jul,31.0,25.0,Mon,9.0,29.0,...,BTRA,1.4,3.5,1,0,3.5,0,3.5,0.08,3.58


In [17]:
bakery_df.dtypes

date             datetime64[ns]
time                     object
year                    float64
MonthN                  float64
Month                    object
WeekN                   float64
Day                     float64
WeekD                    object
H                       float64
Min                     float64
30min                   float64
Rtime                    object
30time                   object
H.5                     float64
Operation                object
Line Item                object
Department               object
Category                 object
Supplier                 object
Supplier Code            object
Cost                     object
Price                    object
Quantity                 object
Modifiers                object
Subtotal                 object
Discounts                object
Net Total                object
Tax                      object
Total Due                object
dtype: object

In [24]:
bakery_df['year'] = bakery_df['year'].fillna(0.0).astype(int)

In [25]:
bakery_df['year'].value_counts()

2017    203373
2018    186308
2016     79201
0            1
Name: year, dtype: int64

In [26]:
bakery_df.loc[bakery_df['year'] == 0, :]

,date,time,year,MonthN,Month,WeekN,Day,WeekD,H,Min,...,Supplier Code,Cost,Price,Quantity,Modifiers,Subtotal,Discounts,Net Total,Tax,Total Due
416580,2018-09-03,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0,0,0,0,0,0,0,0,0


In [29]:
bakery_df = bakery_df.drop(index=416580)
bakery_df.loc[bakery_df['year'] == 0, :]

,date,time,year,MonthN,Month,WeekN,Day,WeekD,H,Min,...,Supplier Code,Cost,Price,Quantity,Modifiers,Subtotal,Discounts,Net Total,Tax,Total Due


In [31]:
bakery_df['MonthN'] = bakery_df['MonthN'].fillna(0.0).astype(int)

In [32]:
bakery_df['MonthN'].value_counts()

9     54096
10    52943
11    48566
8     47183
5     39560
3     36752
6     34988
7     34876
4     33477
12    29130
2     28677
1     28634
Name: MonthN, dtype: int64

In [33]:
bakery_df['WeekN'] = bakery_df['WeekN'].fillna(0.0).astype(int)
bakery_df['WeekN'].value_counts()

46    14343
41    13682
38    13388
39    13320
42    12906
40    12576
45    12565
36    12330
44    11675
37    11619
35    11165
47    10787
19    10562
33    10542
31    10429
34    10353
43    10182
32     9902
20     9035
22     8740
24     8661
21     8426
12     8366
15     8364
23     8333
13     8238
11     8152
25     8148
14     8031
10     7991
49     7987
4      7963
17     7916
16     7768
18     7612
50     7589
5      7530
7      7529
9      7524
29     7517
30     7280
3      7137
8      6994
6      6959
28     6954
27     6876
51     6859
26     6725
52     6211
48     5696
2      5577
53     4109
1      1759
Name: WeekN, dtype: int64

In [34]:
bakery_df['Day'] = bakery_df['Day'].fillna(0.0).astype(int)
bakery_df['Day'].value_counts()

14    17191
13    16874
6     16853
12    16681
11    16531
3     16328
8     16245
17    16059
18    15769
22    15733
9     15727
7     15661
10    15440
19    15419
20    15372
28    15343
16    15308
15    15214
21    15156
5     15108
2     15001
27    14757
29    14622
26    14343
30    14253
4     14007
1     13917
23    13796
25    13739
24    13435
31     9000
Name: Day, dtype: int64

In [35]:
bakery_df.head()

,date,time,year,MonthN,Month,WeekN,Day,WeekD,H,Min,...,Supplier Code,Cost,Price,Quantity,Modifiers,Subtotal,Discounts,Net Total,Tax,Total Due
0,2016-07-25,2016-07-25 09:04:00,2016,7,Jul,31,25,Mon,9.0,4.0,...,MACCHO,1,2.25,1,0,2.25,0,2.25,0.05,2.3
1,2016-07-25,2016-07-25 09:28:00,2016,7,Jul,31,25,Mon,9.0,28.0,...,00:00:00,0,2,2,0,4,0,4,0.09,4.09
2,2016-07-25,2016-07-25 09:28:00,2016,7,Jul,31,25,Mon,9.0,28.0,...,CBC,0,3.75,1,0,3.75,0,3.75,0.38,4.13
3,2016-07-25,2016-07-25 09:28:00,2016,7,Jul,31,25,Mon,9.0,28.0,...,CRC,0,3.8,1,0,3.8,0,3.8,0.09,3.89
4,2016-07-25,2016-07-25 09:29:00,2016,7,Jul,31,25,Mon,9.0,29.0,...,BTRA,1.4,3.5,1,0,3.5,0,3.5,0.08,3.58


In [36]:
cln_bakery_df = bakery_df[['time', 'WeekN', 'WeekD', 'Rtime', 'Operation', 'Line Item', 'Department', \
                           'Category', 'Supplier', 'Supplier Code', 'Cost', 'Price','Quantity', \
                           'Modifiers', 'Subtotal', 'Discounts', 'Net Total', 'Tax', 'Total Due']]

In [37]:
cln_bakery_df.head()

,time,WeekN,WeekD,Rtime,Operation,Line Item,Department,Category,Supplier,Supplier Code,Cost,Price,Quantity,Modifiers,Subtotal,Discounts,Net Total,Tax,Total Due
0,2016-07-25 09:04:00,31,Mon,09:04:00,SALE,Macaron Chocolate,Pastry,general,Cook Au Vin,MACCHO,1,2.25,1,0,2.25,0,2.25,0.05,2.3
1,2016-07-25 09:28:00,31,Mon,09:28:00,SALE,Type in,general,general,not tracked,00:00:00,0,2,2,0,4,0,4,0.09,4.09
2,2016-07-25 09:28:00,31,Mon,09:28:00,SALE,Latte,Coffee,general,Chicago Beverage Company,CBC,0,3.75,1,0,3.75,0,3.75,0.38,4.13
3,2016-07-25 09:28:00,31,Mon,09:28:00,SALE,Chocolate croissant (to go),Croissants,To Go,Cook Au Vin,CRC,0,3.8,1,0,3.8,0,3.8,0.09,3.89
4,2016-07-25 09:29:00,31,Mon,09:29:00,SALE,Baguette tradition,Bread,general,Cook Au Vin,BTRA,1.4,3.5,1,0,3.5,0,3.5,0.08,3.58


In [38]:
cln_bakery_df['Operation'].value_counts()

SALE      468394
RETURN       488
Name: Operation, dtype: int64

In [39]:
cln_bakery_df['Supplier'].value_counts()

not tracked                 238976
Cook Au Vin                 199936
Chicago Beverage Company     15187
French Feast                 13933
Viola Imports                  601
La Colombe                     248
Chef warehouse                   1
Name: Supplier, dtype: int64

In [44]:
bakery_link = "https://www.yelp.com/biz/la-boulangerie-chicago-4?sort_by=date_desc"
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

In [45]:
bakery_response = requests.get(bakery_link)
bakery_soup = bs(bakery_response.text, 'html.parser')

In [46]:
bakery_results = bakery_soup.body.find_all('div', class_='review--with-sidebar')

In [184]:
review_list = []

for review in bakery_results:
    user_info = []
    user_stats = []
    user_name = '' 
    user_loc = ''
    user_friends = ''
    user_reviews = ''
    user_photos = ''
    review_info = []
    review_stars = ''
    review_date = ''
    review_text = ''
    user_check_ins = ''
    elite = ''
    elite_status = ''
    photo = []
    photo_links = []
    
    user_information = {}
    
    try:
        user_info = review.find('ul', class_='user-passport-info')
        user_stats = review.find('ul', class_='user-passport-stats')
        review_info = review.find('div', class_='review-content')
        
        try:
            user_name = user_info.find('li', class_='user-name').find('a').text.strip()
            user_profile_link = user_info.find('a', class_='user-display-name').attrs['href']
            user_profile_link = "https://www.yelp.com" + user_profile_link
            user_loc = user_info.find('li', class_='user-location').find('b').text.strip()
            review_stars = review_info.find('div', class_='i-stars').attrs['title']
            review_stars = int(float(review_stars.split()[0]))
            review_date = review_info.find('span', class_='rating-qualifier').text.strip()
            review_text = review_info.find('p').text
        except AttributeError as error:
            print(error)
            
        try:
            user_friends = user_stats.find('li', class_='friend-count').find('b').text.strip()
        except AttributeError as error:
            if user_stats:
                user_friends = '0'
        
        try:
            user_reviews = user_stats.find('li', class_='review-count').find('b').text.strip()
        except AttributeError as error:
            if user_stats:
                user_reviews = '0'
        
        try:
            user_photos = user_stats.find('li', class_='photo-count').find('b').text.strip()
        except AttributeError as error:
            if user_stats:
                user_photos = '0'
        
        try:
            elite = user_stats.find('li', class_='is-elite')
            if elite:
                elite_status = 'elite'
            else:
                elite_status = 'not elite'
        except AttributeError as error:
            print(error)
        
        try:
            user_check_ins = review_info.find('li', class_='review-tags_item').find('span').text.strip()
            user_check_ins = user_check_ins.split()[0]
        except AttributeError as error:
            if review_info:
                user_check_ins = '0'
        
#         try:
#             photos = review_info.find('ul', class_='photo-box-grid')
#             if photos:
#                 for link in photos:
#                     print(link)
#                     picture = link.find('div', class_='photo-box').find('img').attrs['src']
#                     photo_links.append(picture)
#         except AttributeError as error:
#             if review_info:
#                 photo_links = []

        if (user_name and user_loc and user_friends and user_reviews and user_photos and review_stars \
            and review_date and user_check_ins and user_profile_link and review_text):
            
            user_information = {
                'user_name' : user_name,
                'user_location' : user_loc,
                'user_link' : user_profile_link,
                'user_friends' : int(user_friends),
                'user_reviews' : int(user_reviews),
                'user_photos' : int(user_photos),
                'stars' : review_stars,
                'review_date' : review_date,
                'review_text' : review_text,
#                 'review_photos' : photo_links,
                'check_ins' : int(user_check_ins),
                'elite_status' : elite_status
            }

            review_list.append(user_information)
    except AttributeError as error:
        print(error)

print(json.dumps(review_list, indent=4, sort_keys=True))

'NoneType' object has no attribute 'find'
'NoneType' object has no attribute 'find'
[
    {
        "check_ins": 1,
        "elite_status": "elite",
        "review_date": "12/23/2018",
        "review_text": "I've been dreaming of quiche and croissants as good as the ones I had in Paris. Everything here is phenomenal and baked daily. We've had all types of croissants, muffins, macaroons, and quiche here. My favorite is the salmon and leek quiche. Stop be here, it's a hidden gem. The bread is to die for and smells up the whole bakery.",
        "stars": 5,
        "user_friends": 195,
        "user_link": "https://www.yelp.com/user_details?userid=LlGh8vFtBvix42rxV4pLNQ",
        "user_location": "Chicago, IL",
        "user_name": "Sabrina G.",
        "user_photos": 81,
        "user_reviews": 50
    },
    {
        "check_ins": 0,
        "elite_status": "elite",
        "review_date": "10/10/2018",
        "review_text": "We came to La Boulangerie after recommendation from my cousi